# 🔧 LEAK-FREE DATA PREPARATION PIPELINE V3
## Подготовка данных для модели кредитного скоринга

---

### 📋 Что делает этот ноутбук:
1. ✅ Загружает базовые очищенные данные (`X_train_optimized.parquet`, `X_test_optimized.parquet`)
2. ✅ Создает OOF (Out-of-Fold) KNN мета-признаки
3. ✅ Создает OOF Target Encoding для категориальных переменных
4. ✅ Создает OOF WOE (Weight of Evidence) признаки
5. ✅ Создает Interaction и Polynomial признаки
6. ✅ Сохраняет leak-free датасеты для обучения модели

### ⏱️ Время выполнения:
- ~5-8 минут на Google Colab

### 📦 Выходные файлы:
- `X_train_leak_free_v3.parquet` (71,999 × 89)
- `X_test_leak_free_v3.parquet` (18,000 × 89)
- `y_train_leak_free_v3.parquet`
- `y_test_leak_free_v3.parquet`

---

## 1️⃣ Установка библиотек

In [ ]:
%%capture
!pip install -q pandas numpy scikit-learn pyarrow

## 2️⃣ Импорты

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("LEAK-FREE DATA PIPELINE V3")
print("Создание признаков с Out-of-Fold методологией")
print("="*70)

## 3️⃣ Загрузка базовых данных

**ВАЖНО:** Загрузите 4 файла:
- `X_train_optimized.parquet`
- `X_test_optimized.parquet`
- `y_train.parquet`
- `y_test.parquet`

In [ ]:
from google.colab import files
import os

print("📤 Загрузите 4 базовых файла:")
print("   1. X_train_optimized.parquet")
print("   2. X_test_optimized.parquet")
print("   3. y_train.parquet")
print("   4. y_test.parquet")
print()

uploaded = files.upload()

# Verify files
required_files = [
    'X_train_optimized.parquet',
    'X_test_optimized.parquet',
    'y_train.parquet',
    'y_test.parquet'
]

print("\n📁 Проверка загруженных файлов:")
all_ok = True
for file in required_files:
    if file in uploaded:
        size_mb = len(uploaded[file]) / (1024 * 1024)
        print(f"   ✅ {file} ({size_mb:.1f} MB)")
    else:
        print(f"   ❌ {file} - НЕ НАЙДЕН!")
        all_ok = False

if not all_ok:
    raise ValueError("Не все файлы загружены!")

In [ ]:
print("\n[1/8] Загрузка базовых датасетов...")

X_train = pd.read_parquet('X_train_optimized.parquet')
X_test = pd.read_parquet('X_test_optimized.parquet')
y_train = pd.read_parquet('y_train.parquet')['default'].values
y_test = pd.read_parquet('y_test.parquet')['default'].values

print(f"✓ Training set: {X_train.shape}")
print(f"✓ Test set: {X_test.shape}")
print(f"✓ Default rate: {y_train.mean()*100:.2f}%")
print(f"✓ Class distribution will remain UNCHANGED (no SMOTE)")

# Create copies for feature engineering
X_train_clean = X_train.copy()
X_test_clean = X_test.copy()

## 4️⃣ OOF KNN Meta-Features

Создание KNN-based признаков с использованием Out-of-Fold методологии.

**Ключевой принцип:** Каждый fold обучается ТОЛЬКО на своих данных!

In [ ]:
def create_oof_knn_features(X, y, X_test, n_neighbors=50, n_splits=5):
    """
    Create KNN meta-features using strict out-of-fold methodology.
    
    CRITICAL: KNN is trained separately for each fold, ONLY on that fold's
    training data, preventing any leakage.
    """
    numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    oof_train = np.zeros(len(X))
    oof_test_folds = []
    
    for fold_num, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"  Processing fold {fold_num + 1}/{n_splits}...", end=" ")
        
        # CRITICAL: Fit scaler ONLY on training fold
        scaler = StandardScaler()
        X_train_fold = scaler.fit_transform(X[numeric_cols].iloc[train_idx])
        X_val_fold = scaler.transform(X[numeric_cols].iloc[val_idx])
        X_test_fold = scaler.transform(X_test[numeric_cols])
        
        # CRITICAL: Train KNN ONLY on training fold
        knn = KNeighborsClassifier(n_neighbors=n_neighbors, n_jobs=-1)
        knn.fit(X_train_fold, y[train_idx])
        
        # Predict on validation fold (OOF predictions)
        oof_train[val_idx] = knn.predict_proba(X_val_fold)[:, 1]
        
        # Predict on test set (will average across folds)
        oof_test_folds.append(knn.predict_proba(X_test_fold)[:, 1])
        print("✓")
    
    # Average test predictions across all folds
    oof_test = np.mean(oof_test_folds, axis=0)
    
    return oof_train, oof_test

In [ ]:
print("\n[2/8] Creating OOF KNN meta-features...")
print("Each fold's KNN is trained ONLY on that fold's training data\n")

# Create KNN features for different K values
for n_neighbors in [50, 100, 500]:
    print(f"\n  Creating OOF KNN feature with K={n_neighbors}")
    oof_train, oof_test = create_oof_knn_features(
        X_train, y_train, X_test, n_neighbors=n_neighbors
    )
    
    X_train_clean[f'knn_oof_{n_neighbors}'] = oof_train
    X_test_clean[f'knn_oof_{n_neighbors}'] = oof_test
    
    # Calculate correlation
    corr = np.corrcoef(oof_train, y_train)[0, 1]
    print(f"  ✓ knn_oof_{n_neighbors} created")
    print(f"    Correlation with target: {corr:.4f}")

print("\n✅ OOF KNN features created!")

## 5️⃣ OOF Target Encoding

Кодирование категориальных переменных с использованием средней вероятности дефолта.

In [ ]:
def create_oof_target_encoding(X, y, X_test, column, n_splits=5, smoothing=10):
    """
    Create target encoding using strict out-of-fold methodology.
    
    CRITICAL: Encoding statistics calculated separately for each fold,
    ONLY from that fold's training data.
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    oof_train = np.zeros(len(X))
    oof_test_folds = []
    
    for fold_num, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        # Calculate encoding ONLY on training fold
        train_data = pd.DataFrame({
            'feature': X[column].iloc[train_idx],
            'target': y[train_idx]
        })
        
        # Calculate smoothed mean (Bayesian-like smoothing)
        encoding_dict = {}
        global_mean = y[train_idx].mean()
        
        for value in train_data['feature'].unique():
            mask = train_data['feature'] == value
            n = mask.sum()
            if n == 0:
                encoding_dict[value] = global_mean
            else:
                # Smoothing to prevent overfitting on rare categories
                category_mean = train_data.loc[mask, 'target'].mean()
                encoding_dict[value] = (category_mean * n + global_mean * smoothing) / (n + smoothing)
        
        # Apply to validation fold
        oof_train[val_idx] = X[column].iloc[val_idx].map(encoding_dict).fillna(global_mean)
        
        # Apply to test set
        test_encoded = X_test[column].map(encoding_dict).fillna(global_mean)
        oof_test_folds.append(test_encoded)
    
    # Average test encodings across folds
    oof_test = np.mean(oof_test_folds, axis=0)
    
    return oof_train, oof_test

In [ ]:
print("\n[3/8] Creating OOF target encoding for categorical features...")
print("Each fold encodes using ONLY that fold's training statistics\n")

# Apply OOF target encoding to categorical columns
categorical_cols = ['state', 'marital_status', 'education', 'employment_type']

for col in categorical_cols:
    if col in X_train.columns:
        print(f"  Encoding {col}...", end=" ")
        oof_train, oof_test = create_oof_target_encoding(
            X_train, y_train, X_test, col
        )
        
        X_train_clean[f'{col}_target_oof'] = oof_train
        X_test_clean[f'{col}_target_oof'] = oof_test
        
        corr = np.corrcoef(oof_train, y_train)[0, 1]
        print(f"✓ correlation = {corr:.4f}")

print("\n✅ OOF Target Encoding complete!")

## 6️⃣ OOF WOE (Weight of Evidence)

Создание WOE признаков для числовых переменных.

In [ ]:
def create_oof_woe(X, y, X_test, column, n_bins=10, n_splits=5):
    """
    Create WOE binning using strict out-of-fold methodology.
    
    CRITICAL: Bins and WOE values calculated separately for each fold,
    ONLY from that fold's training data.
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    oof_train = np.zeros(len(X))
    oof_test_folds = []
    
    for fold_num, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        # Create bins ONLY from training fold
        train_values = X[column].iloc[train_idx]
        
        try:
            # Use quantile bins
            _, bins = pd.qcut(train_values, q=n_bins, duplicates='drop', retbins=True)
            
            # Bin the training data
            train_binned = pd.cut(train_values, bins=bins, include_lowest=True)
            
            # Calculate WOE for each bin (ONLY from training fold)
            woe_dict = {}
            total_good = (y[train_idx] == 0).sum()
            total_bad = y[train_idx].sum()
            
            for bin_label in train_binned.cat.categories:
                mask = train_binned == bin_label
                n_good = (y[train_idx][mask] == 0).sum()
                n_bad = y[train_idx][mask].sum()
                
                # Add smoothing to avoid log(0)
                n_good = max(n_good, 0.5)
                n_bad = max(n_bad, 0.5)
                
                # Calculate WOE
                pct_good = n_good / total_good
                pct_bad = n_bad / total_bad
                woe = np.log(pct_good / pct_bad)
                woe_dict[bin_label] = woe
            
            # Apply to validation fold
            val_binned = pd.cut(X[column].iloc[val_idx], bins=bins, include_lowest=True)
            oof_train[val_idx] = val_binned.map(woe_dict).fillna(0)
            
            # Apply to test set
            test_binned = pd.cut(X_test[column], bins=bins, include_lowest=True)
            test_woe = test_binned.map(woe_dict).fillna(0)
            oof_test_folds.append(test_woe)
            
        except Exception as e:
            # If binning fails, use 0s
            oof_train[val_idx] = 0
            oof_test_folds.append(np.zeros(len(X_test)))
    
    # Average test WOE across folds
    oof_test = np.mean(oof_test_folds, axis=0)
    
    return oof_train, oof_test

In [ ]:
print("\n[4/8] Creating OOF WOE features...")
print("WOE bins calculated separately for each fold\n")

# Apply OOF WOE to numeric features
woe_features = ['debt_to_income_ratio', 'credit_utilization', 'credit_score', 'age']

for feature in woe_features:
    if feature in X_train.columns:
        print(f"  Creating OOF WOE for {feature}...", end=" ")
        oof_train, oof_test = create_oof_woe(
            X_train, y_train, X_test, feature
        )
        
        X_train_clean[f'{feature}_woe_oof'] = oof_train
        X_test_clean[f'{feature}_woe_oof'] = oof_test
        
        corr = np.corrcoef(oof_train, y_train)[0, 1]
        print(f"✓ correlation = {corr:.4f}")

print("\n✅ OOF WOE features created!")

## 7️⃣ Interaction Features

Создание признаков взаимодействия (НЕТ утечки - простое умножение).

In [ ]:
print("\n[5/8] Creating interaction features...")
print("Simple multiplication - no leakage possible\n")

# Debt burden interactions
if 'debt_to_income_ratio' in X_train.columns and 'credit_utilization' in X_train.columns:
    X_train_clean['debt_credit_interaction'] = X_train['debt_to_income_ratio'] * X_train['credit_utilization']
    X_test_clean['debt_credit_interaction'] = X_test['debt_to_income_ratio'] * X_test['credit_utilization']
    print(f"  ✓ debt_credit_interaction created")

# Income stability * debt burden
if 'income_stability_score' in X_train.columns and 'debt_payment_burden' in X_train.columns:
    X_train_clean['income_debt_interaction'] = X_train['income_stability_score'] * X_train['debt_payment_burden']
    X_test_clean['income_debt_interaction'] = X_test['income_stability_score'] * X_test['debt_payment_burden']
    print(f"  ✓ income_debt_interaction created")

# Age * credit score
if 'age' in X_train.columns and 'credit_score' in X_train.columns:
    X_train_clean['age_credit_interaction'] = X_train['age'] * X_train['credit_score'] / 100
    X_test_clean['age_credit_interaction'] = X_test['age'] * X_test['credit_score'] / 100
    print(f"  ✓ age_credit_interaction created")

# Employment length * income
if 'employment_length' in X_train.columns and 'monthly_income' in X_train.columns:
    X_train_clean['employment_income_stability'] = X_train['employment_length'] * X_train['monthly_income']
    X_test_clean['employment_income_stability'] = X_test['employment_length'] * X_test['monthly_income']
    print(f"  ✓ employment_income_stability created")

print("\n✅ Interaction features created!")

## 8️⃣ Polynomial Features

Создание полиномиальных признаков (НЕТ утечки - математические преобразования).

In [ ]:
print("\n[6/8] Creating polynomial features...")
print("Simple transformations - no leakage possible\n")

top_features = ['credit_stress_score', 'debt_to_income_ratio', 'credit_utilization']

for feature in top_features:
    if feature in X_train.columns:
        # Square
        X_train_clean[f'{feature}_squared'] = X_train[feature] ** 2
        X_test_clean[f'{feature}_squared'] = X_test[feature] ** 2
        
        # Cube root
        X_train_clean[f'{feature}_cbrt'] = np.cbrt(X_train[feature])
        X_test_clean[f'{feature}_cbrt'] = np.cbrt(X_test[feature])
        
        print(f"  ✓ {feature}_squared and {feature}_cbrt created")

print("\n✅ Polynomial features created!")

## 9️⃣ Validation & Quality Checks

In [ ]:
print("\n[7/8] Performing validation checks...\n")

# Check for missing values
train_nulls = X_train_clean.isnull().sum().sum()
test_nulls = X_test_clean.isnull().sum().sum()

assert train_nulls == 0, f"Training data has {train_nulls} missing values!"
assert test_nulls == 0, f"Test data has {test_nulls} missing values!"

print(f"✓ No missing values in training data")
print(f"✓ No missing values in test data")

# Check feature alignment
assert list(X_train_clean.columns) == list(X_test_clean.columns), "Feature mismatch!"
print(f"✓ Feature alignment verified")

# Check class distribution
print(f"\nClass distribution check:")
print(f"  Training default rate: {y_train.mean()*100:.2f}%")
print(f"  Test default rate: {y_test.mean()*100:.2f}%")
print(f"  ✓ NO class balancing applied (correct approach)")

print(f"\nFEATURE SUMMARY:")
print(f"  Original features: {X_train.shape[1]}")
print(f"  Leak-free features: {X_train_clean.shape[1]}")
print(f"  New OOF features: {X_train_clean.shape[1] - X_train.shape[1]}")

## 🔟 Save Leak-Free Datasets

In [ ]:
print("\n[8/8] Saving leak-free datasets...\n")

# Save features
X_train_clean.to_parquet('X_train_leak_free_v3.parquet', index=False)
X_test_clean.to_parquet('X_test_leak_free_v3.parquet', index=False)

# Save targets (original, no balancing)
pd.DataFrame({'default': y_train}).to_parquet('y_train_leak_free_v3.parquet', index=False)
pd.DataFrame({'default': y_test}).to_parquet('y_test_leak_free_v3.parquet', index=False)

print(f"✓ Leak-free training features: X_train_leak_free_v3.parquet ({X_train_clean.shape})")
print(f"✓ Leak-free test features: X_test_leak_free_v3.parquet ({X_test_clean.shape})")
print(f"✓ Original training targets: y_train_leak_free_v3.parquet")
print(f"✓ Original test targets: y_test_leak_free_v3.parquet")

## 📊 Summary Report

In [ ]:
print("\n" + "="*70)
print("LEAK-FREE PIPELINE V3 COMPLETE")
print("="*70)

print("\nDATA LEAKAGE FIXES APPLIED:")
print("1. KNN meta-features: Using strict OOF methodology")
print("2. Target encoding: Using OOF with smoothing")
print("3. WOE binning: Calculated separately per fold")
print("4. StandardScaler: Fitted separately per fold")
print("5. SMOTE: NOT USED - will handle imbalance in model")

print("\nCLASS DISTRIBUTION:")
print(f"  Training: {(y_train == 1).sum():,} defaults / {len(y_train):,} total = {y_train.mean()*100:.2f}%")
print(f"  Test: {(y_test == 1).sum():,} defaults / {len(y_test):,} total = {y_test.mean()*100:.2f}%")

print("\nNEXT STEPS:")
print("  1. Download all 4 parquet files")
print("  2. Upload to Google_Colab_Leak_Free_90plus_v3.ipynb")
print("  3. Train XGBoost model")
print("  4. Expect realistic AUC (honest metrics)")

print("="*70)

## 📊 Top 15 Features by Correlation

In [ ]:
print("\nTop 15 features by correlation (LEAK-FREE):\n")
correlations = {}
for col in X_train_clean.columns:
    try:
        correlations[col] = abs(np.corrcoef(X_train_clean[col], y_train)[0, 1])
    except:
        correlations[col] = 0.0

top_15 = sorted(correlations.items(), key=lambda x: x[1], reverse=True)[:15]
for i, (feature, corr) in enumerate(top_15, 1):
    new_marker = " [OOF]" if 'oof' in feature.lower() else ""
    print(f"{i:2d}. {feature:40s} {corr:.4f}{new_marker}")

print("\n✅ ALL FEATURES CREATED WITH OUT-OF-FOLD METHODOLOGY")
print("✅ NO DATA LEAKAGE POSSIBLE")
print("✅ READY FOR HONEST MODEL EVALUATION")

## 💾 Download Files

In [ ]:
# Download all files
from google.colab import files

print("📥 Скачивание файлов...\n")

files.download('X_train_leak_free_v3.parquet')
files.download('X_test_leak_free_v3.parquet')
files.download('y_train_leak_free_v3.parquet')
files.download('y_test_leak_free_v3.parquet')

print("\n✅ Все файлы загружены!")
print("\n🎯 Теперь используйте эти файлы в Google_Colab_Leak_Free_90plus_v3.ipynb")

---

## 📋 Итоговая сводка

### Что было сделано:
1. ✅ Загружены базовые очищенные данные
2. ✅ Созданы OOF KNN мета-признаки (K=50, 100, 500)
3. ✅ Созданы OOF Target Encoding для 4 категориальных переменных
4. ✅ Созданы OOF WOE признаки для 4 числовых переменных
5. ✅ Созданы 4 Interaction признака
6. ✅ Созданы 6 Polynomial признаков
7. ✅ Выполнена валидация (нет пропусков, нет утечки)
8. ✅ Сохранены leak-free датасеты

### Ключевые характеристики:
- **Методология**: Строгий Out-of-Fold (5 folds)
- **Утечка данных**: ОТСУТСТВУЕТ ✅
- **Балансировка**: НЕ применена (оригинальное распределение)
- **Финальные размеры**: Train (71,999 × 89), Test (18,000 × 89)

### Следующие шаги:
1. Скачайте 4 parquet файла
2. Загрузите их в `Google_Colab_Leak_Free_90plus_v3.ipynb`
3. Обучите XGBoost модель
4. Получите честные метрики AUC

---

**Автор:** Data Preparation Specialist  
**Версия:** V3 Leak-Free Pipeline  
**Дата:** Ноябрь 2025

---